<a href="https://colab.research.google.com/github/FranciscoOcampoPredictiva/azureml_course/blob/main/Hyperdrive_Training_Script.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Import the class
from azureml.core import Run

In [ ]:
# Get the run context
new_run = Run.get_context()

In [ ]:
# Access the workspace
ws = new_run.experiment.workspace

In [ ]:
# Get the parameters
import argparse

parser = argparse.ArgumentParser()
parser.add_argument('--n_estimators', type=int)
parser.add_argument('--min_samples_leaf', type=int)
parser.add_argument('--input-data', type=str)

args = parser.parse_args()
ne = args.n_estimators
msl = args.min_samples_leaf

In [ ]:
# read the input dataset
import pandas as pd
df = new_run.input_datasets['raw_data'].to_pandas_dataframe()

In [ ]:
# Select relevant columns from the dataset
dataPrep = df.drop(columns=['RowNumber', 'CustomerId', 'Surname'], axis=1)

In [ ]:
# Create Dummy variables / encoding the categorical variables
dataPrep = pd.get_dummies(dataPrep, drop_first=True)

In [ ]:
# Define X and Y
X = dataPrep.drop(['Exited'], axis=1)
Y = dataPrep[['Exited']]

In [ ]:
# Split the dataset into train and test set
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.3, random_state = 0, stratify=Y)

In [ ]:
# Build and train the Random Forest Classification model
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(n_estimators=ne, min_samples_leaf=msl)
classifier.fit(X_train, Y_train)

In [ ]:
# Predict the output - Scored Label
Y_predict = classifier.predict(X_test)

In [ ]:
# Scored Probabilities
Y_prob = classifier.predict_proba(X_test)[:, 1]

In [ ]:
# Confusion matrix and accuracy score
from sklearn.metrics import confusion_matrix
cm    = confusion_matrix(Y_test, Y_predict)
score = classifier.score(X_test, Y_test)

In [ ]:
new_run.log('accuracy', score)

In [ ]:
new_run.complete()